In [1]:
%%html
<style>
div.jp-OutputArea-output pre {
    white-space: pre;
}
</style>

In [2]:
from time import sleep
from pyspark.sql.types import StringType,IntegerType
from pyspark.sql.functions import *
from pyspark.sql.window import *
from pyspark.storagelevel import StorageLevel
from pyspark.sql import SparkSession

# # O spark session é fundamental para a execução do script no cluster, ele deve ter esse mesmo formato.
# # Altere o valor do app_name para o nome do seu job
app_name = "Análise de Volumetria"
spark = (SparkSession
         .builder
         .appName(app_name)
         .config("spark.yarn.dist.file", "/etc/spark/conf.dist/hive-site.xml")
         .config("hive.metastore.client.factory.class", "com.amazonaws.glue.catalog.metastore.AWSGlueDataCatalogHiveClientFactory")
         .enableHiveSupport()
         .getOrCreate()
        )

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
6798,application_1664549316508_3342,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df = spark.sql('''
select 
      *
from 
     app_privacy.privacy_metrics
''')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
unpivotExpr = f'''
stack(
13,
'name',cast(`name` as string),
'addresses',cast(`addresses` as string),
'birth_date',cast(`birth_date` as string),
'documents',cast(`documents` as string),
'email',cast(`email` as string),
'mother_name',cast(`mother_name` as string),
'father_name',cast(`father_name` as string),
'gender',cast(`gender` as string),
'phone_number',cast(`phone_number` as string),
'selfie',cast(`selfie` as string),
'cnh_front',cast(`cnh_front` as string),
'cnh_verse',cast(`cnh_verse` as string),
'federativeunit',cast(`federativeunit` as string)
) as (item,count)
'''

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
unPivotDF = df.select("people_id", expr(unpivotExpr))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
df_final = (unPivotDF
            .groupBy('item')
            .agg(
                sum('count').cast('bigint').alias('total')
            )
            .orderBy('item')
           )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
df_final.show(15,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…